# BOLETIM DO COMÉRCIO EXTERIOR


## Variáveis Globais

In [1]:
nome_planilha = 'BCE 2024.03 - São João del-Rei'
nome_guia     = 'EXPORTAÇÃO - Países'

## Importando bibliotecas

In [2]:
# General
import pandas as pd

# Graphs
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Google
import gspread
from google.colab import auth
from google.auth import default

## Autenticando no Google Planilhas

In [3]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## Importando dados das planilhas

In [4]:
# Selecionando a planilha que está os dados
planilha = gc.open(nome_planilha)

# Selecionando a guia em que os dados estão
guia = planilha.worksheet(nome_guia)

# Obtendo todos os dados dessa guia em uma lista de valores
dados = guia.get_all_values()

# Convertendo a lista de 'dados' para um DataFrame
sjdr_exports = pd.DataFrame(dados[1:], columns=dados[0])

In [5]:
# Vizualizar o dataframe
sjdr_exports.head()

,,,,,,
0,,País,Valor (US$),,País,Valor (US$)
1,,Estônia,"$ 3.355.488,00",,Outros,"$ 6.237.035,00"
2,,Índia,"$ 3.114.103,00",,Índia,"$ 3.114.103,00"
3,,Estados Unidos,"$ 2.830.754,00",,Estados Unidos,"$ 2.830.754,00"
4,,Japão,"$ 1.965.590,00",,Japão,"$ 1.965.590,00"


### Formatando os dados

In [6]:
# Definir a primeira linha como cabeçalho
sjdr_exports.columns = sjdr_exports.iloc[0]
sjdr_exports = sjdr_exports.iloc[1:].reset_index(drop=True)

# Deletar colunas que não vou utilizar
sjdr_exports = sjdr_exports.iloc[0:, :3]

# Remover a primeira coluna
sjdr_exports = sjdr_exports.drop(sjdr_exports.columns[0], axis=1)

# Renomear colunas
sjdr_exports = sjdr_exports.rename(columns={
    sjdr_exports.columns[0]: 'country',
    sjdr_exports.columns[1]: 'value',
})

# Substituir valores por vazio
sjdr_exports['value'] = sjdr_exports['value'].str.replace(',00', '', regex=True)
sjdr_exports['value'] = sjdr_exports['value'].str.replace('\$', '', regex=True)
sjdr_exports['value'] = sjdr_exports['value'].str.replace('.', '', regex=False)

# Remover espaços em branco e caracteres não numéricos
sjdr_exports['value'] = sjdr_exports['value'].str.replace('[^\d.]', '', regex=True).str.strip()

# Converter os dados para números (float)
sjdr_exports['value'] = sjdr_exports['value'].astype(float)

In [7]:
# Vizualizar informações do dataframe para comparar
sjdr_exports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  35 non-null     object 
 1   value    35 non-null     float64
dtypes: float64(1), object(1)
memory usage: 688.0+ bytes


### Tabela com código dos países

In [8]:
planilha = gc.open('Lista com códigos dos países')

guia = planilha.worksheet('Lista')

dados = guia.get_all_values()

# Converter 'dados' para um DataFrame
countries = pd.DataFrame(dados[1:], columns=dados[0])

# Deletar colunas
countries.drop(['country', 'alpha-2-code', 'numeric'], axis=1, inplace=True)

# Renomear colunas
countries = countries.rename(columns={
    countries.columns[0]: 'country',
    countries.columns[1]: 'code',
})

### Mesclando os dados para plotar

In [9]:
# Mesclando os dados de exportação com códigos dos países
exports = pd.merge(countries, sjdr_exports, how='inner', on='country', indicator=False)

In [10]:
# Vizualizar o dataframe
exports.head()

,country,code,value
0,África do Sul,ZAF,75373.0
1,Alemanha,DEU,50613.0
2,Argentina,ARG,1320082.0
3,Austrália,AUS,180727.0
4,Áustria,AUT,144862.0


In [11]:
# Vizualizar informações do dataframe
exports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  35 non-null     object 
 1   code     35 non-null     object 
 2   value    35 non-null     float64
dtypes: float64(1), object(2)
memory usage: 968.0+ bytes


## Plotando o gráfico

In [12]:
data = go.Choropleth(
    locations = exports['code'],
    z = exports['value'],
    text = exports['country'],
    colorscale = 'blugrn',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = 'U$',
    colorbar_title = 'Valores<br> em Milhões (U$)',
    showlegend=True,
    showscale=True,
)

layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig = go.Figure(data=data, layout=layout)

fig = fig.update_layout(
    font_family="Arial",
    font_color="#FFFFFF", # 004651
    title_font_family="Arial",
    title_font_color="#FFFFFF",
    legend_title_font_color="#FFFFFF",
    width=900,
    height=500,
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='equirectangular',
        bgcolor= '#F4F4F0',
        landcolor='rgba(206,206,206,1)',
    ),
    margin={"r":0,"t":0,"l":0,"b":0},
)

fig.update_geos(fitbounds="locations", visible=True)

fig.show()